* Fabrycio Leite Nakano Almada.
Instituto de Informática, Universidade Federal de Goiás
fabrycio@discente.ufg.br
* Kauan Divino Pouso Mariano.
Instituto de Informática, Universidade Federal de Goiás
kauan@discente.ufg.br
* Maykon Adriell Dutra.
Instituto de Informática, Universidade Federal de Goiás
maykonadriell@discente.ufg.br
* Victor Emanuel da Silva Monteiro.
Instituto de Informática, Universidade Federal de Goiás
victor_emanuel@discente.ufg.br


Este código implementa um sistema de suporte emocional contínuo que integra processamento de áudio, análise de sentimentos e geração de respostas empáticas, tudo isso auxiliado por um banco de dados robusto e uma interface de usuário amigável. O sistema é projetado para ser utilizado junto com o acompanhamento de um psicólogo, oferecendo uma ferramenta que permite aos pacientes registrar e gerenciar suas emoções em tempo real, mesmo fora das sessões de terapia.

## importações

In [ ]:
# Instalar dependências
!pip install accelerate bitsandbytes transformers git+https://github.com/openai/whisper.git
!pip install librosa==0.9.0 pydub noisereduce scikit-learn tensorflow torchaudio seaborn matplotlib sqlalchemy smtplib
!pip install -U funasr modelscope addict datasets==2.18.0 simplejson gradio
!sudo apt update && sudo apt install ffmpeg

  Cloning https://github.com/openai/whisper.git to /tmp/pip-req-build-uewwf_6g
  Running command git clone --filter=blob:none --quiet https://github.com/openai/whisper.git /tmp/pip-req-build-uewwf_6g
  Resolved https://github.com/openai/whisper.git to commit ba3f3cd54b0e5b8ce1ab3de13e32122d0d5f98ab
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 314.1/314.1 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 119.8/119.8 MB 7.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 46.8 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manyli

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Hit:7 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:10 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:11 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [858 kB]
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1,410 kB]
Get:13 http://archive.ubuntu.com/ubun

In [ ]:
!pip install config

In [ ]:
!pip install modelscope

In [ ]:
import os
import zipfile
import torch
import librosa
import torchaudio
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, f1_score
from sklearn.model_selection import train_test_split
from transformers import pipeline, AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
import logging
from modelscope.pipelines import pipeline as modelscope_pipeline
from modelscope.utils.constant import Tasks
from sqlalchemy import create_engine, Column, String, Integer, Text, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship
from sqlalchemy.exc import IntegrityError
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
import json
import gradio as gr
import config
from dotenv import load_dotenv
import gradio as gr
from sqlalchemy import create_engine, Column, Integer, String, Text, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker
import logging
import os
import smtplib
from email.mime.multipart import MIMEMultipart
from email.mime.text import MIMEText
from transformers import pipeline
import requests

## configurações

In [ ]:
load_dotenv()
HF_TOKEN = os.getenv("HF_TOKEN")
DATABASE_URL = "sqlite:///database.db"
EMAIL_FROM = os.getenv("EMAIL_FROM")
EMAIL_PASSWORD = os.getenv("EMAIL_PASSWORD")
Base = declarative_base()


<ipython-input-2-d928416d652f>:14: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  Base = declarative_base()


## banco de dados

In [ ]:
class Psicologo(Base):
    __tablename__ = 'psicologos'
    id = Column(Integer, primary_key=True)
    nome = Column(String, unique=True)
    email = Column(String, unique=True)

class Paciente(Base):
    __tablename__ = 'pacientes'
    id = Column(Integer, primary_key=True)
    nome = Column(String)
    psicologo_id = Column(Integer, ForeignKey('psicologos.id'))
    psicologo = relationship("Psicologo", back_populates="pacientes")

Psicologo.pacientes = relationship("Paciente", order_by=Paciente.id, back_populates="psicologo")

class Conversa(Base):
    __tablename__ = 'conversas'
    id = Column(Integer, primary_key=True)
    paciente_id = Column(Integer, ForeignKey('pacientes.id'))
    texto = Column(Text)
    paciente = relationship("Paciente", back_populates="conversas")

Paciente.conversas = relationship("Conversa", order_by=Conversa.id, back_populates="paciente")

engine = create_engine(DATABASE_URL)
Base.metadata.create_all(engine)
Session = sessionmaker(bind=engine)

No sistema de banco de dados, são armazenadas informações detalhadas sobre psicólogos, pacientes e suas conversas. Psicólogos têm seus nomes e e-mails registrados, garantindo unicidade para facilitar a identificação. Pacientes são associados a seus respectivos psicólogos, o que permite uma fácil consulta e acompanhamento. As conversas entre pacientes e o sistema são armazenadas cronologicamente, permitindo um rastreamento eficiente das interações ao longo do tempo. Esse esquema facilita a gestão e recuperação de dados essenciais para o funcionamento do sistema de suporte emocional.

## modelo

In [ ]:
# chamadas API do ChatGPT
def ask_chatgpt(prompt, system_prompt="Você é um assistente psicológico virtual especializado em emergências psicológicas."):
    url = "https://api.openai.com/v1/chat/completions"
    headers = {
        "Authorization": "Bearer sk-proj-apikey",
        "Content-Type": "application/json"
    }
    data = {
        "model": "gpt-3.5-turbo",
        "messages": [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ]
    }
    response = requests.post(url, headers=headers, data=json.dumps(data))
    response_data = response.json()

    if 'choices' in response_data:
        return response_data['choices'][0]['message']['content']
    else:
        print("Erro na resposta da API OpenAI:", response_data)
        return "Erro ao processar a solicitação com o modelo GPT-3.5."

O sistema utiliza uma função especializada para interagir com a API do ChatGPT, que fornece respostas baseadas em prompts específicos. O modelo é configurado para agir como um assistente psicológico, respondendo de maneira empática e contextualizada. Esta função é usada para gerar respostas que ajudam no suporte emocional contínuo, garantindo que os pacientes recebam orientações rápidas e pertinentes.

## processamento de áudio

In [ ]:
class ProcessadorAudio:
    def __init__(self, taxa_amostragem_alvo=16000, n_fft=1024, hop_length=512, n_mels=64, max_pad_len=400):
        self.taxa_amostragem_alvo = taxa_amostragem_alvo
        self.n_fft = n_fft
        self.hop_length = hop_length
        self.n_mels = n_mels
        self.max_pad_len = max_pad_len

    def carregar_audio(self, caminho):
        waveform, sample_rate = torchaudio.load(caminho)
        if sample_rate != self.taxa_amostragem_alvo:
            resample_transform = torchaudio.transforms.Resample(orig_freq=sample_rate, new_freq=self.taxa_amostragem_alvo)
            waveform = resample_transform(waveform)
        return waveform

    def extrair_caracteristicas(self, waveform):
        mel_spectrogram_transform = torchaudio.transforms.MelSpectrogram(
            sample_rate=self.taxa_amostragem_alvo,
            n_fft=self.n_fft,
            hop_length=self.hop_length,
            n_mels=self.n_mels
        )
        mel_spectrogram = mel_spectrogram_transform(waveform)
        mel_spectrogram = mel_spectrogram.mean(dim=0)
        pad = self.max_pad_len - mel_spectrogram.shape[-1]
        if pad > 0:
            mel_spectrogram = torch.nn.functional.pad(mel_spectrogram, (0, pad))
        else:
            mel_spectrogram = mel_spectrogram[:, :self.max_pad_len]
        return mel_spectrogram.numpy().flatten()

class DetectorEmocao:
    def __init__(self):
        self.modelo = modelscope_pipeline(task=Tasks.emotion_recognition, model="iic/emotion2vec_plus_large")
        self.labels_mapping = {
            '生气/angry': 'angry',
            '开心/happy': 'happy',
            '中立/neutral': 'neutral',
            '难过/sad': 'sad',
            '<unk>': 'unknown',
        }

    def prever(self, caminho_audio):
        rec_result = self.modelo(caminho_audio, granularity="utterance", extract_embedding=False)
        for result in rec_result:
            predicted_emotion = self.labels_mapping[result['labels'][np.argmax(result['scores'])]]
            return predicted_emotion

O processamento de áudio começa com o carregamento e a preparação dos arquivos de áudio. Os áudios são ajustados para uma taxa de amostragem padrão e convertidos em espectrogramas Mel, que destacam as frequências presentes de maneira perceptualmente relevante. Este processamento inicial inclui a resampling (ajuste da taxa de amostragem), a transformação de Fourier (STFT) e a criação de espectrogramas Mel, que são essenciais para a análise precisa das emoções contidas no áudio. O espectrograma Mel é padronizado para um comprimento fixo para garantir consistência nos dados de entrada. Essas features extraídas são então utilizadas por um modelo especializado para identificar emoções como alegria, tristeza, raiva e neutralidade, permitindo uma compreensão detalhada do estado emocional dos pacientes a partir de suas mensagens de áudio.

## processamento NLP

In [ ]:
class ProcessadorNLP:
    def __init__(self):
        self.analisador_sentimento = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")

    def analisar_sentimento(self, texto):
        resultado = self.analisador_sentimento(texto)
        sentimento = resultado[0]['label']
        mapeamento_sentimentos = {
            "1 star": "sad",
            "2 stars": "neutral",
            "3 stars": "neutral",
            "4 stars": "happy",
            "5 stars": "happy"
        }
        return mapeamento_sentimentos.get(sentimento, "neutral")

Para compreender o conteúdo textual das mensagens, o sistema utiliza o modelo BERT base-multilingual-uncased, que é um dos modelos de linguagem mais robustos e versáteis disponíveis. Este modelo foi treinado em uma ampla gama de textos em diversos idiomas, permitindo uma compreensão profunda e precisa do conteúdo emocional das mensagens. Textos transcritos dos áudios são analisados para identificar sentimentos predominantes, como felicidade, neutralidade ou tristeza. O modelo classifica o sentimento do texto em categorias como "sad", "neutral" e "happy". Essa análise ajuda a determinar o estado emocional do paciente com base no texto, garantindo que as respostas fornecidas sejam apropriadas e empáticas.

## serviço email

In [ ]:
def enviar_email(psicologo_email, assunto, conteudo):
    msg = MIMEMultipart()
    msg['From'] = EMAIL_FROM
    msg['To'] = psicologo_email
    msg['Subject'] = assunto

    msg.attach(MIMEText(conteudo, 'plain'))

    try:
        server = smtplib.SMTP('smtp.gmail.com:587')
        server.starttls()
        server.login(EMAIL_FROM, EMAIL_PASSWORD)
        server.send_message(msg)
        server.quit()
        print("Email enviado com sucesso!")
    except Exception as e:
        print(f"Ocorreu um erro ao enviar o email: {e}. O processo continuará.")


A comunicação com psicólogos é facilitada por um serviço de email automatizado. Relatórios detalhados das interações dos pacientes são gerados e enviados diretamente aos psicólogos, permitindo um acompanhamento contínuo. Este processo garante que os psicólogos estejam sempre informados sobre o progresso emocional de seus pacientes, auxiliando na gestão do tratamento de maneira eficiente.

## assistente

In [ ]:
class AssistenteAudioEmocional:
    def __init__(self):
        self.processador_audio = ProcessadorAudio()
        self.detector_emocao = DetectorEmocao()
        self.processador_nlp = ProcessadorNLP()
        self.session = Session()
        self.historico = []

    def processar_audio(self, caminho_audio):
        logging.info("Processando áudio...")
        return self.detector_emocao.prever(caminho_audio)

    def transcrever_audio(self, caminho_audio):
        logging.info("Transcrevendo áudio...")
        transcricao_path = caminho_audio.replace('.wav', '.txt')
        os.system(f'whisper {caminho_audio} --model medium --task transcribe --language pt --output_dir /tmp/gradio/')
        transcricao_path = os.path.join("/tmp/gradio", os.path.basename(caminho_audio).replace('.wav', '.txt'))
        with open(transcricao_path, 'r') as file:
            transcricao = file.read()
        return transcricao

    def processar_texto(self, texto):
        logging.info("Processando texto...")
        return self.processador_nlp.analisar_sentimento(texto)

    def gerar_resposta(self, emocao_audio, emocao_texto, transcricao):
        logging.info("Gerando resposta...")
        self.historico.append(f"Paciente: {transcricao}")
        self.historico = self.historico[-4:]  # Manter apenas as últimas duas interações
        prompt = (f"Histórico: {' '.join(self.historico)}\n"
                  f"Transcrição do áudio: {transcricao} \n"
                  f"Emoção detectada no áudio: {emocao_audio} \n"
                  f"Emoção detectada no texto transcrito: {emocao_texto} \n"
                  f"Baseado na ciência psicológica, gere uma resposta para este paciente! Seja empático e tome cuidado com as palavras. Forneça respostas curtas, objetivas e completas. Não dê respostas pela metade. Lembre-se que o paciente está vulnerável e precisa de respostas rápidas e precisas. Responda de forma humanizada, não demonstre que é um chatbot. Lembre que você é um auxiliar do psicólogo, mas não substitui um. A resposta deve ter um parágrafo somente. NÃO CORTE AS RESPOSTAS. CONCLUA SEMPRE O RACIOCÍNIO. RESPONDA COMO UM HUMANO.")
        resposta = ask_chatgpt(prompt)
        self.historico.append(f"Assistente: {resposta}")
        return resposta

    def salvar_conversa(self, paciente_id, texto):
        conversa = Conversa(paciente_id=paciente_id, texto=texto)
        self.session.add(conversa)
        self.session.commit()

    def enviar_relatorio(self, paciente_id):
        paciente = self.session.query(Paciente).filter_by(id=paciente_id).first()
        psicologo = paciente.psicologo
        conversas = self.session.query(Conversa).filter_by(paciente_id=paciente_id).all()
        historico = "\n".join([c.texto for c in conversas])

        assunto = f"Relatório da Análise de Sentimentos do Paciente {paciente.nome}"
        conteudo = f"Histórico da conversa:\n{historico}\n\nResumo da conversa:\n{self.resumir_conversa(historico)}"
        enviar_email(psicologo.email, assunto, conteudo)

    def resumir_conversa(self, historico):
        prompt = f"Resuma o seguinte histórico de conversa, destacando as emoções mais presentes e os assuntos mais pertinentes em bullet points, não invente, apenas perceba e traga as emoções mais citadas, como se fosse o relatório resumido:\n\n{historico}\n\nResumo:"
        resumo = ask_chatgpt(prompt)
        return resumo

    def lidar_interacao(self, caminho_audio=None, texto=None, paciente_id=None):
        if caminho_audio:
            emocao_audio = self.processar_audio(caminho_audio)
            transcricao = self.transcrever_audio(caminho_audio)
            emocao_texto = self.processar_texto(transcricao)
            resposta = self.gerar_resposta(emocao_audio, emocao_texto, transcricao)
        elif texto:
            emocao_texto = self.processar_texto(texto)
            resposta = self.gerar_resposta('unknown', emocao_texto, texto)
        else:
            resposta = "Nenhum áudio ou texto fornecido."

        if paciente_id:
            self.salvar_conversa(paciente_id, f"Paciente: {texto or transcricao}\nAssistente: {resposta}")

        return resposta

    def gerar_relatorio(self):
        conversas = self.session.query(Conversa).all()
        if not conversas:
            historico = "\n".join(self.historico)
        else:
            historico = "\n".join([c.texto for c in conversas])

        resumo = self.resumir_conversa(historico)

        relatorio = f"Relatório da Análise de Sentimentos\n\n"
        relatorio += f"Resumo da conversa:\n{resumo}\n\n"
        relatorio += f"Histórico completo da conversa:\n{historico}"

        return relatorio

O assistente emocional integra todos os componentes do sistema, processando áudios e textos, analisando emoções e gerando respostas. Ele mantém um registro histórico das interações para fornecer respostas contextualmente apropriadas. Além disso, o assistente pode gerar relatórios das conversas e enviá-los por email aos psicólogos, facilitando o acompanhamento contínuo do estado emocional dos pacientes. Essa integração de funcionalidades permite um suporte emocional abrangente.

Além de suas funções técnicas, o assistente foi cuidadosamente projetado com um foco específico nos prompts utilizados para gerar respostas. O prompt passado ao modelo GPT-3.5 é estruturado para garantir que as respostas sejam empáticas, humanizadas e completas. O objetivo é que o assistente responda de forma que os pacientes se sintam compreendidos e apoiados. Instruções claras são fornecidas ao modelo para evitar respostas fragmentadas e garantir que o raciocínio seja sempre concluído, promovendo uma interação que mimetiza o cuidado e a atenção de um profissional humano, sem substituir a função do psicólogo.

## main

In [ ]:
assistente = AssistenteAudioEmocional()

def chat_interface(audio=None, text=None, paciente_id=None):
    if audio is not None:
        resposta = assistente.lidar_interacao(caminho_audio=audio, paciente_id=paciente_id)
    elif text is not None:
        resposta = assistente.lidar_interacao(texto=text, paciente_id=paciente_id)
    else:
        resposta = "Por favor, forneça áudio ou texto para continuar."
    return resposta

def launch_interface():
    with gr.Blocks() as demo:
        gr.Markdown("# Assistente Psicológico Virtual")
        with gr.Row():
            audio_input = gr.Audio(type="filepath")
            text_input = gr.Textbox(lines=2, placeholder="Digite sua mensagem aqui...")
        paciente_id = gr.Number(label="ID do Paciente", precision=0)
        output = gr.Textbox(label="Resposta do Assistente")
        submit_btn = gr.Button("Enviar")
        report_btn = gr.Button("Gerar Relatório")

        submit_btn.click(chat_interface, inputs=[audio_input, text_input, paciente_id], outputs=output)
        report_btn.click(generate_report, inputs=[], outputs=gr.Textbox(label="Relatório"))

    demo.launch(share=True, debug=True)

def generate_report():
    report = assistente.gerar_relatorio()
    return report

if __name__ == "__main__":
    launch_interface()

2024-07-11 07:37:03,032 - modelscope - WARNING - Model revision not specified, use revision: v2.0.5
2024-07-11 07:37:03,604 - modelscope - INFO - initiate model from /root/.cache/modelscope/hub/iic/emotion2vec_plus_large
2024-07-11 07:37:03,606 - modelscope - INFO - initiate model from location /root/.cache/modelscope/hub/iic/emotion2vec_plus_large.
2024-07-11 07:37:03,608 - modelscope - INFO - initialize model from /root/.cache/modelscope/hub/iic/emotion2vec_plus_large


You are using the latest version of funasr-1.1.0
Detect model requirements, begin to install it: /root/.cache/modelscope/hub/iic/emotion2vec_plus_large/requirements.txt
install model requirements successfully
Warning, miss key in ckpt: modality_encoders.AUDIO.decoder.blocks.0.0.weight, /root/.cache/modelscope/hub/iic/emotion2vec_plus_large/model.pt
Warning, miss key in ckpt: modality_encoders.AUDIO.decoder.blocks.0.0.bias, /root/.cache/modelscope/hub/iic/emotion2vec_plus_large/model.pt
Warning, miss key in ckpt: modality_encoders.AUDIO.decoder.blocks.1.0.weight, /root/.cache/modelscope/hub/iic/emotion2vec_plus_large/model.pt
Warning, miss key in ckpt: modality_encoders.AUDIO.decoder.blocks.1.0.bias, /root/.cache/modelscope/hub/iic/emotion2vec_plus_large/model.pt
Warning, miss key in ckpt: modality_encoders.AUDIO.decoder.blocks.2.0.weight, /root/.cache/modelscope/hub/iic/emotion2vec_plus_large/model.pt
Warning, miss key in ckpt: modality_encoders.AUDIO.decoder.blocks.2.0.bias, /root/.ca

2024-07-11 07:37:12,549 - modelscope - WARNING - No preprocessor field found in cfg.
2024-07-11 07:37:12,553 - modelscope - WARNING - No val key and type key found in preprocessor domain of configuration.json file.
2024-07-11 07:37:12,556 - modelscope - WARNING - Cannot find available config to build preprocessor at mode inference, current config: {'model_dir': '/root/.cache/modelscope/hub/iic/emotion2vec_plus_large'}. trying to build by task and model information.
2024-07-11 07:37:12,560 - modelscope - WARNING - No preprocessor key ('funasr', 'emotion-recognition') found in PREPROCESSOR_MAP, skip building preprocessor.


Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
Running on public URL: https://1e425223eba7ca6747.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


Keyboard interruption in main thread... closing server.
Killing tunnel 127.0.0.1:7860 <> https://1e425223eba7ca6747.gradio.live


A aplicação principal configura e lança uma interface de usuário amigável, permitindo que pacientes interajam facilmente com o assistente emocional. Os usuários podem enviar áudios ou textos e receber respostas imediatas. A interface também permite a geração de relatórios detalhados das conversas, que são úteis para os psicólogos no acompanhamento do tratamento. Essa interface acessível garante que os pacientes recebam suporte emocional contínuo e imediato, contribuindo significativamente para seu bem-estar.